In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input, GRU
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from models_def import Attention

max_features = 180000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
# 
word_vec_dict = {}
with open('../crawl-300d-2M.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.rstrip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')

EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

Using TensorFlow backend.


(159571, 6)
(159571, 150) (153164, 150)
yarne 91537
athonite 55455
gbagbo 123757
calender 27493
collaborateur 36307
fd8baaaaebaj 143551
2000000
Preparing embedding matrix
Null word embeddings: 79399


In [2]:
from sklearn.metrics import log_loss,accuracy_score
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU,CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
import gc


def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNGRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    att = Attention(maxlen)(x)
    conc = concatenate([att,avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [3]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_model()
        batch_size = 64
        epochs = 8
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), 
                  callbacks=callbacks_list)
        
        model.load_weights(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


print('def done')

def done


In [4]:
import pickle
sample_submission = pd.read_csv("../input/sample_submission.csv")

train_pred,test_pred = kf_train(fold_cnt=10,rnd=4)
print(train_pred.shape,test_pred.shape)    


# 40000, 150
# final 0.040348153796 0.984148122153, pub 9849
# 180000,150, concat add attention, 10 fold
# final 0.0389350573515 0.984694378469， PUB 9861

Train on 143613 samples, validate on 15958 samples
Epoch 1/8
143613/143613 [==============================] - 120s 836us/step - loss: 0.0560 - acc: 0.9801 - val_loss: 0.0418 - val_acc: 0.9837
Epoch 2/8
143613/143613 [==============================] - 114s 796us/step - loss: 0.0418 - acc: 0.9837 - val_loss: 0.0405 - val_acc: 0.9844
Epoch 3/8
143613/143613 [==============================] - 117s 812us/step - loss: 0.0392 - acc: 0.9845 - val_loss: 0.0390 - val_acc: 0.9848
Epoch 4/8
143613/143613 [==============================] - 117s 813us/step - loss: 0.0371 - acc: 0.9855 - val_loss: 0.0386 - val_acc: 0.9850
Epoch 5/8
143613/143613 [==============================] - 120s 837us/step - loss: 0.0355 - acc: 0.9860 - val_loss: 0.0397 - val_acc: 0.9843
Epoch 6/8
143613/143613 [==============================] - 118s 818us/step - loss: 0.0337 - acc: 0.9866 - val_loss: 0.0392 - val_acc: 0.9846
Epoch 7/8
143613/143613 [==============================] - 118s 819us/step - loss: 0.0323 - acc: 0.9872

In [5]:
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/pool_gru1_fasttext_adj1_sample_10.gz", index=False, compression='gzip')
with open('../features/pool_gru_fasttext_adj1_10_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print(sample_submission.head())
print('===================================')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.997501      0.550919  0.979196  0.171820  0.934243   
1  0000247867823ef7  0.000349      0.000013  0.000144  0.000014  0.000171   
2  00013b17ad220c46  0.000716      0.000049  0.000303  0.000060  0.000304   
3  00017563c3f7919a  0.000130      0.000008  0.000089  0.000076  0.000095   
4  00017695ad8997eb  0.003955      0.000048  0.000337  0.000099  0.000421   

   identity_hate  
0       0.575691  
1       0.000014  
2       0.000120  
3       0.000007  
4       0.000027  
